In [2]:
import tifffile
import rasterio
import geopandas as gpd
from shapely import wkt
from rasterio.vrt import WarpedVRT
import pandas as pd
import numpy as np
from pyproj import Transformer

## Nitrate data

In [3]:
nitrate_path = '/Users/Administrator/Documents/University/Year 3/2b/Thesis/Bachelor-Thesis/data/aligned/merged_dataset_1.csv'
nitrate_df = pd.read_csv(nitrate_path)

In [4]:
nitrate_df["geometry"] = nitrate_df["geometry"].apply(wkt.loads)
gdf = gpd.GeoDataFrame(nitrate_df, geometry="geometry", crs="EPSG:4326")

In [5]:
gdf = gdf.to_crs("EPSG:3035")

gdf['date'] = pd.to_datetime(gdf['date'], errors='coerce', utc=True)
gdf['year'] = gdf['date'].dt.year

# Filter for years 2008–2014
mask_years = gdf["year"].between(2008, 2014)
gdf_sub = gdf.loc[mask_years].copy()

## Ph values

In [6]:
raster_path = "/Users/Administrator/Documents/University/Year 3/2b/Thesis/Bachelor-Thesis/data/raw/soil_ph_cacl2/ph.cacl2_iso.10390.2021.index_m_30m_b0cm..20cm_20080101_20101231_eu_epsg.3035_v20250212.tif"


In [7]:
with tifffile.TiffFile(raster_path) as tif:
    for tag in tif.pages[0].tags.values():
        print(tag.name, tag.value)

ImageWidth 216700
ImageLength 151552
BitsPerSample 8
Compression COMPRESSION.ADOBE_DEFLATE
PhotometricInterpretation PHOTOMETRIC.MINISBLACK
SamplesPerPixel 1
PlanarConfiguration PLANARCONFIG.CONTIG
Predictor PREDICTOR.HORIZONTAL
TileWidth 2048
TileLength 2048
TileOffsets (827401992, 827410145, 827418298, 827426451, 827434604, 827442757, 827450910, 827459063, 827467216, 827475369, 827483522, 827491675, 827499828, 827507981, 827516134, 827524287, 827532440, 827540593, 827548746, 827556899, 827565052, 827573205, 827581358, 827589511, 827597664, 827605817, 827613970, 827622123, 827630276, 827638429, 827646582, 827654735, 827662888, 827671041, 827679194, 827687347, 827695500, 827703653, 827711806, 827719959, 827728112, 827736265, 827744418, 827752571, 827760724, 827787543, 827805861, 827814014, 827822167, 827830320, 827838473, 827846626, 827854779, 827862932, 827871085, 827879238, 827887391, 827895544, 827903697, 827911850, 827920003, 827928156, 827936309, 827944462, 827952615, 828053847, 8

In [8]:
with rasterio.open(raster_path) as src:
    coords = [(geom.x, geom.y) for geom in gdf_sub.geometry]
    rows_cols = [src.index(x, y) for x, y in coords]
    pixel_vals = src.read(1)[tuple(zip(*rows_cols))]

# Apply scale and NoData mask
nodata = 255
scale = 0.1
ph_vals = np.where(pixel_vals != nodata, pixel_vals * scale, np.nan)

# Overwrite 'acidity_1' for 2008–2014 with valid pH values
gdf.loc[mask_years, "acidity_1"] = ph_vals

In [11]:
gdf.drop(columns=["year"]).to_csv("updated_dataset.csv", index=False)